In [1]:
import pandas as pd
import time
from dotenv import load_dotenv
load_dotenv("./.env.local")
import os
BEARER_TOKEN = os.getenv("BEARER_TOKEN")
import tweepy
client = tweepy.Client(BEARER_TOKEN, wait_on_rate_limit=True)
from pathlib import Path

In [3]:
# Preprocessing the datasets and save them in csv files under csvdataframes folder
Path.mkdir(Path("csvdataframes"), exist_ok=True)
dataset_path = Path("dataset")

for idx, day in enumerate(dataset_path.iterdir()):
    full_dataset = pd.read_csv(day, compression='gzip')
    df_en = full_dataset[full_dataset['language']=='en']
    df_en_filteted = df_en[["userid", "tweetid", "text", "hashtags"]]
    df_no_duplicate = df_en_filteted.drop_duplicates(subset='text', keep='first')
    df_sampled = df_no_duplicate.sample(1)
    df_sampled.to_csv(Path(f"csvdataframes/day_{idx}.csv"))

C:\Users\Bao\AppData\Local\Temp\ipykernel_10852\3492015089.py:6: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  full_dataset = pd.read_csv(day, compression='gzip')
C:\Users\Bao\AppData\Local\Temp\ipykernel_10852\3492015089.py:6: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  full_dataset = pd.read_csv(day, compression='gzip')
C:\Users\Bao\AppData\Local\Temp\ipykernel_10852\3492015089.py:6: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  full_dataset = pd.read_csv(day, compression='gzip')
C:\Users\Bao\AppData\Local\Temp\ipykernel_10852\3492015089.py:6: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  full_dataset = pd.read_csv(day, compression='gzip')
C:\Users\Bao\AppData\Local\Temp\ipykernel_10852\3492015089.py:6: DtypeWarning: Columns (15) have mixed types. Specify dt

In [4]:
# Method to call on each csv day files
def create_edgelist(df_sample):  
#     edgelist_retweet = open(f"edgelists/edgelist_retweet_{time.time()}.txt", "a")
    edgelist_retweet = open(f"edgelists/edgelist_retweet.txt", "a")
    edgelist_retweet.write("tweet_id rt_user_id\n")

#     edgelist_like = open(f"edgelists/edgelist_like_{time.time()}.txt", "a")
    edgelist_like = open(f"edgelists/edgelist_likes.txt", "a")
    
    edgelist_like.write("tweet_id like_user_id\n")

    for index, row in df_sample.iterrows():
        tweetid = row['tweetid']
        print(f"{tweetid=}")
        tweet = client.get_tweet(id=tweetid, expansions='referenced_tweets.id')
        
        # Deleted tweets are still in the dataset => just skip them for now
        if (tweet.errors):
            continue
            
        og_tweet_id = tweet.includes.get('tweets')[0].id if tweet.includes.get('tweets') else tweetid

        # Get users who retweeted
        users_retweet = client.get_retweeters(id=og_tweet_id)
        
        # Skip if no retweet
        if not users_retweet.data:
            continue
            
        for user in users_retweet.data:
            edgelist_retweet.write(f"{og_tweet_id} {user.id}\n")
            continue

        next_token = users_retweet.meta.get('next_token')

        while(next_token):
            users_retweet = client.get_retweeters(id=og_tweet_id, pagination_token=next_token)
            
            if not users_retweet.data:
                continue
            
            for user in users_retweet.data:
                edgelist_retweet.write(f"{og_tweet_id} {user.id}\n")
                
            next_token = users_retweet.meta.get('next_token')

        # Get users who liked
        users_like = client.get_liking_users(id=og_tweet_id)
        
        # Skip if no like
        if not users_like.data:
            continue
        
        for user in users_like.data:
            edgelist_like.write(f"{og_tweet_id} {user.id}\n")
            
        next_token = users_like.meta.get('next_token')

        while(next_token):
            users_like = client.get_retweeters(id=og_tweet_id, pagination_token=next_token)
            
            if not users_like.data:
                continue
            
            for user in users_like.data:
                edgelist_like.write(f"{og_tweet_id} {user.id}\n")
                
            next_token = users_like.meta.get('next_token')

    edgelist_retweet.close()
    edgelist_like.close()

In [ ]:
# Main loop
Path.mkdir(Path("edgelists"), exist_ok=True)
for file in Path("csvdataframes").iterdir():
    df = pd.read_csv(file)
    create_edgelist(df)

tweetid=1497766842580021248


Rate limit exceeded. Sleeping for 888 seconds.
Rate limit exceeded. Sleeping for 888 seconds.
Rate limit exceeded. Sleeping for 888 seconds.
Rate limit exceeded. Sleeping for 888 seconds.
Rate limit exceeded. Sleeping for 889 seconds.
Rate limit exceeded. Sleeping for 889 seconds.
Rate limit exceeded. Sleeping for 889 seconds.
Rate limit exceeded. Sleeping for 889 seconds.
